In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 1000 # number of expression blocks
num_data = 9100   # number of data
test_size = 100   # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 50                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 250300.11
Epoch 1, Validation Loss: 105608.96
Epoch 2, Validation Loss: 47803.63
Epoch 3, Validation Loss: 96714.10
Epoch 4, Validation Loss: 29958.46
Epoch 5, Validation Loss: 18415.31
Epoch 6, Validation Loss: 32887.87
Epoch 7, Validation Loss: 15332.06
Epoch 8, Validation Loss: 17650.53
Epoch 9, Validation Loss: 65983.40
Epoch 10, Validation Loss: 12805.18
Epoch 11, Validation Loss: 89033.76
Epoch 12, Validation Loss: 13231.24
Epoch 13, Validation Loss: 29147.78
Epoch 14, Validation Loss: 64409.50
Epoch 15, Validation Loss: 31453.57
Epoch 16, Validation Loss: 9241.98
Epoch 17, Validation Loss: 21350.22
Epoch 18, Validation Loss: 18845.12
Epoch 19, Validation Loss: 10293.02
Epoch 20, Validation Loss: 47123.09
Epoch 21, Validation Loss: 26911.31
Epoch 22, Validation Loss: 8702.21
Epoch 23, Validation Loss: 20163.75
Epoch 24, Validation Loss: 82046.79
Epoch 25, Validation Loss: 8134.00
Epoch 26, Validation Loss: 8740.09
Epoch 27, Validation Loss: 8572.42
Epoch

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-1000_g.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  7.89it/s]


           Obj Val  Constraints Viol  Elapsed Time
count   100.000000             100.0    100.000000
mean   7276.955921               0.0      0.004163
std     734.736728               0.0      0.000838
min    5815.733168               0.0      0.002570
25%    6748.714360               0.0      0.003511
50%    7460.260706               0.0      0.004095
75%    7703.307959               0.0      0.004741
max    9909.261714               0.0      0.006168
Number of infeasible solution: 0


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 50                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 226581.63
Epoch 1, Validation Loss: 75214.19
Epoch 2, Validation Loss: 63800.46
Epoch 3, Validation Loss: 14408.81
Epoch 4, Validation Loss: 16327.14
Epoch 5, Validation Loss: 24660.89
Epoch 6, Validation Loss: 16997.48
Epoch 7, Validation Loss: 40456.14
Epoch 8, Validation Loss: 87106.26
Epoch 9, Validation Loss: 21422.06
Epoch 10, Validation Loss: 47217.79
Epoch 11, Validation Loss: 10129.52
Epoch 12, Validation Loss: 10835.40
Epoch 13, Validation Loss: 18685.11
Epoch 14, Validation Loss: 15587.62
Epoch 15, Validation Loss: 28501.72
Epoch 16, Validation Loss: 15411.71
Epoch 17, Validation Loss: 8575.90
Epoch 18, Validation Loss: 7984.38
Epoch 19, Validation Loss: 17027.51
Epoch 20, Validation Loss: 13328.68
Epoch 21, Validation Loss: 8642.40
Epoch 22, Validation Loss: 8213.72
Epoch 23, Validation Loss: 8648.14
Epoch 24, Validation Loss: 32994.19
Epoch 25, Validation Loss: 10724.45
Epoch 26, Validation Loss: 35373.94
Epoch 27, Validation Loss: 10172.16
Epoch 

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-1000_g.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  7.74it/s]


           Obj Val  Constraints Viol  Elapsed Time
count   100.000000             100.0    100.000000
mean   5780.845894               0.0      0.004389
std     816.747961               0.0      0.001057
min    4458.487083               0.0      0.002517
25%    5143.012673               0.0      0.003646
50%    5590.293859               0.0      0.004328
75%    6422.063858               0.0      0.004996
max    8017.801037               0.0      0.008850
Number of infeasible solution: 0
